# Quick Tour

---

This is a quick tour of `pybuck`---a package to support dimensional analysis in Python.

In [1]:
from pybuck import *


We are studying the *Reynolds' pipe flow problem* [1], which has physical inputs:

| Input | Symbol | Units |
|--|--|--|
| Fluid density | $\rho$ | $\frac{M}{L^3}$ |
| Fluid bulk velocity | $U$ | $\frac{L}{T}$ |
| Pipe diameter | $D$ | $L$ |
| Fluid dynamic viscosity | $\mu$ | $\frac{M}{LT}$ |
| Roughness lengthscale | $\epsilon$ | $L$ |

We construct the *dimension matrix* [2] with simple calls:

In [2]:
df_dim = col_matrix(
    rho = dict(M=1, L=-3),
    U   = dict(L=1, T=-1),
    D   = dict(L=1),
    mu  = dict(M=1, L=-1, T=-1),
    eps = dict(L=1)
)
df_dim


,rowname,rho,U,D,mu,eps
0,M,1,0,0,1,0
1,T,0,-1,0,-1,0
2,L,-3,1,1,-1,1


This is a standardized way to represent the physical units of the inputs.

We can compute a basis for the nullspace of `df_dim`; this subspace is called the *pi subspace*.

In [7]:
df_pi = pi_basis(df_dim)
df_pi


,rowname,pi0,pi1
0,rho,-0.527504,0.086337
1,U,-0.527504,0.086337
2,D,-0.377964,-0.654654
3,mu,0.527504,-0.086337
4,eps,-0.149539,0.740990


This shows us that the pi subspace is two-dimensional. The weights are fairly non-standard; a more familiar set are the *Reynolds number* and the *relative roughness*.

In [5]:
df_standard = col_matrix(
    Re = dict(rho=1, U=1, D=1, mu=-1), # Reynold number
    R  = dict(eps=1, D=-1)             # Relative roughness
)
df_standard


,rowname,Re,R
0,eps,0,1
1,D,1,-1
2,mu,-1,0
3,rho,1,0
4,U,1,0


We can check that these two bases are the same by computing the *subspace angles* between them:

In [8]:
angles(df_standard, df_pi)

array([3.13563382e-16, 1.65653949e-16])

All subspace angles are zero to working precision; these two bases express the same subspace.

Since these are the same subspace, we can re-express the computed basis `df_pi` in terms of the standard basis `df_standard`. This will allow us to physically interpret the computationally-derived weights.

In [9]:
df_pi_prime = express(df_pi, df_standard)
df_pi_prime


,rowname,pi0,pi1
0,Re,-0.527504,0.086337
1,R,-0.149539,0.740990


This shows us that `pi0` is composed of both `Re` and `R`, while `pi1` is almost entirely aligned with `

# Bibliography

---

[1] O. Reynolds, "An experimental investigation of the circumstances which determine whether the motion of water shall be direct or sinuous, and of the law of resistance in parallel channels" (1883) *Royal Society*

[2] Z. del Rosario, M. Lee, and G. Iaccarino, "Lurking Variable Detection via Dimensional Analysis" (2019) *SIAM/ASA Journal on Uncertainty Quantification*